## Prompting Pipeline for comparison of different prompting techniques

In [1]:
import pandas as pd
import numpy as np
import pickle
import mlflow
import mlflow.pyfunc
import sacrebleu

### Data Loading

In [3]:
# Daten laden
text = pd.read_pickle('machine_translation.pkl')
text

# Standardisierung ?

,complexity,text_german,text_english
0,easy,Felix hat es satt: Ständig ist Mama unterwegs....,Felix is fed up: Mom is always on the go. But ...
1,news_gen,Die rund 1.400 eingesetzten Beamten haben demn...,"The approximately 1,400 deployed officers have..."
2,news_spec,"Der Staatschef hat zugleich aber das Recht, vo...",The head of state also has the right to appoin...
3,pop_science,Dass der Klimawandel die Hitzewellen in Südasi...,There is no question that climate change is in...
4,science,"Der DSA-110, der sich am Owens Valley Radio Ob...","The DSA-110, situated at the Owens Valley Radi..."


### Prompt Composition

In [ ]:
def create_prompt(text, mode="zero-shot"):
    if mode == "zero-shot":
        return f"Translate this text into German: {text}"
    elif mode == "few-shot":
        examples = "Example 1: Hello -> Hallo\nExample 2: Goodbye -> Auf Wiedersehen\n"
        return f"{examples}Translate this text into German: {text}"

### Model Interaction

In [ ]:
# Modell laden
translation_model = pipeline("translation_en_to_de", model="lmstudio-ai/gemma-2b-it-GGUF")

# Prompt senden und Ergebnisse sammeln
def translate_text(prompt):
    result = translation_model(prompt)
    return result[0]["translation_text"]

### Evaluation

In [4]:
import pandas as pd
import hashlib
from llama_cpp import Llama

# Daten laden
def load_data():
    data = pd.read_pickle("machine_translation.pkl")
    return data

# Prompts erstellen
def generate_prompts(data):
    prompts = []
    for _, row in data.iterrows():
        text = row['text_english']
        cleaned_text = text.replace("\n", " ").strip()
        prompt = f"Translate the following text to German: {cleaned_text}"
        prompts.append(prompt)
    data['prompt'] = prompts
    return data

# Übersetzungen mit llama.cpp durchführen
def translate_with_llama(data):
    translations = []

    with mlflow.start_run(run_name="Translation with llama.cpp"):
        mlflow.log_param("model", "gemma-2b-it-GGUF")

        for index, row in data.iterrows():
            prompt = row['prompt']
            response = gemma(prompt, max_tokens=512, echo=False, stop=["Q:"])
            translated_text = response['choices'][0]['text']

            # Metriken loggen
            mlflow.log_metric(f"translation_length_{index}", len(translated_text))

            # Generate a unique filename for logging, using a hash of the prompt
            prompt_hash = hashlib.md5(prompt.encode("utf-8")).hexdigest()[:8]
            translated_text_hash = hashlib.md5(translated_text.encode("utf-8")).hexdigest()[:8]
            mlflow.log_text(f"Prompt_{index}", prompt_hash)
            mlflow.log_text(f"Translation_{index}", translated_text_hash)

            translations.append(translated_text)

        # Ergebnisse als Artefakt speichern
        data['translation'] = translations
        data.to_csv("translations.csv", index=False)
        mlflow.log_artifact("translations.csv")

    return data

# Hauptpipeline
def main_pipeline():
    # Daten laden
    data = load_data()
    print("Daten geladen.")

    # Prompts erstellen
    data = generate_prompts(data)
    print("Prompts erstellt.")

    # Übersetzungen durchführen
    data = translate_with_llama(data)
    print("Übersetzungen durchgeführt.")

    # Ergebnisse speichern
    output_path = "translated_results.csv"
    data.to_csv(output_path, index=False)
    print(f"Ergebnisse gespeichert: {output_path}")

# Ausführung
experiment_name = "Pipeline_FirstTest"
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(experiment_name)

mlflow.set_experiment(experiment_name)

gemma = Llama.from_pretrained(
	repo_id="lmstudio-ai/gemma-2b-it-GGUF",
	filename="gemma-2b-it-q8_0.gguf",
    echo=False,
    stop=["Q:"],
    n_gpu_layers=1,
    verbose=False,
    )

main_pipeline()

llama_new_context_with_model: n_ctx_per_seq (512) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


Daten geladen.
Prompts erstellt.
🏃 View run Translation with llama.cpp at: http://127.0.0.1:5000/#/experiments/227834415347659719/runs/5aca6312e6d24af8a7239327286fbd4d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/227834415347659719
Übersetzungen durchgeführt.
Ergebnisse gespeichert: translated_results.csv
